In [1]:
import numpy as np
import pickle

In [3]:
path_to_ci = "BBQ_results_test/CI_after_localization_round_2_cc_0.1_lt2_0.0_width_50_std_2.pickle"
loci, loci_lists = pickle.load(open(path_to_ci,"rb"))
print(len(loci), len(loci_lists))

3 3


In [ ]:
new_dir = path_to_ci.split(".")[0]+"_orf_and_anno"
print(new_dir)
try:
    os.mkdir(new_dir)
except:
    print(f"already have directory {new_dir}")

In [ ]:
loci

In [ ]:
loci_list

In [ ]:
for i,li in enumerate(loci_lists):
    z = " ".join([str(_) for _ in li])
    ! python /n/desai_lab/users/efenton/bbq/yeast_info/get_orf.py {z} > {new_dir}/orf_out_{i}.txt

In [ ]:
all_loci = [i for j in loci_lists for i in j]
loci_to_orf_d = dict.fromkeys(all_loci)
for i,li in enumerate(loci_lists):
    orf_file = f"{new_dir}/orf_out_{i}.txt"
    print(orf_file)
    fh = open(orf_file, 'r')
    success_loci= []
    orfs = []
    loci_time = False
    orf_time = False
    for line in fh.readlines():
        if loci_time:
            new_loci = line.split()
            if new_loci[0]=='[]':
                print("none annotated")
                break
            #print(new_loci)
            if new_loci[0][0]=='[':
                if new_loci[0]=='[':
                    new_loci = new_loci[1:]
                else:
                    new_loci[0] = new_loci[0][1:]
            if new_loci[-1][-1]==']':
                new_loci[-1] = new_loci[-1][:-1]
                loci_time = False
            #print(new_loci)
            success_loci += [int(_) for _ in new_loci]
        if orf_time:
            orfs = [x[7:-1] for x in line.split(',')[:-1]] + [line.split(',')[-1][7:-3]]
        if line.startswith("Named"):
            loci_time = True
        if line.startswith("Gene"):
            orf_time = True
    if len(orfs)!= len(success_loci):
        raise ValueError("different number of ORFs and loci...something went wrong")
    fh.close()
    #print(orfs, success_loci)
    for _, loci in enumerate(success_loci):
       # print(loci, orfs[_])
        loci_to_orf_d[loci] = orfs[_] 

In [ ]:
loci_to_possible_orfs_d = {}
all_possible_orfs = []
all_orfs = []
for ind in range(num_to_localize):
    l = idx_filt[ind]
    ls = loci_filt[ind]
    plt.figure(figsize = (20,10))
    plt.imshow(to_plot[ind], vmin = -.03, vmax = .03, cmap ="bwr_r")
    plt.title(ls, fontsize = 20)
    plt.show()
    for _ in sorted(loci_lists[ind]):
        print(loci_to_orf_d[_], _)
    possible_orfs = set([loci_to_orf_d[_] for _ in loci_lists[ind]])
    possible_orfs = [ _ for _ in possible_orfs if _ is not None]
    all_possible_orfs.append(possible_orfs)
    all_orfs+=possible_orfs
    print("\n")
    print(possible_orfs)
    loci_to_possible_orfs_d[ls]= possible_orfs

In [ ]:
plt.scatter(range(num_to_localize), [len(_) for _ in all_possible_orfs], alpha= .5)

In [ ]:
plt.hist([len(_) for _ in all_possible_orfs[:30]], bins = np.arange(0,10,1))
plt.title("top 30 loci")
plt.xlabel("number of ORFs")

In [ ]:
## Save annotations and adjcent info

fh = open(f"{new_dir}/orf_names.txt", "w")
fh.write(" ".join(all_orfs))
fh.close()

pickle.dump(loci_to_possible_orfs_d, open(f"{new_dir}/loci_to_possible_orfs_d", "wb"))
#loci_to_possible_orfs_d = pickle.load(open(f"{new_dir}/loci_to_possible_orfs_d", "rb"))

# upload file for process and function annotations here: https://www.yeastgenome.org/goSlimMapper
# had to remove ambigous results-> gofriendly
# save tab delinated results to 
fun_anno = f"{new_dir}/go_annotations_function.txt"
proc_anno = f"{new_dir}/go_annotations_process.txt"

# use https://yeastmine.yeastgenome.org/yeastmine/begin.do to get annotations for orf_names.txt 
# ignored ones with ambigous labels
# save tab delinated results to 
gene_anno = f"{new_dir}/gene_anno.tsv"

all_orfs = []
for key in loci_to_possible_orfs_d:
    all_orfs += loci_to_possible_orfs_d[key]
print(len(all_orfs))
all_orfs = list(set(all_orfs))
print(len(all_orfs))

df = pd.read_csv(gene_anno, sep = '\t', header=None)#, cols = ["name1", "name2", "func","description"])
df = df.set_index(0)
df.rename(columns = {0:'Systematic Gene Name', 1:'Standard Gene Name',2:'Long Gene Name',3:'Brief Description',4:'Description', 5:'Function Summary'}, inplace = True)


for orf in all_orfs:
    if orf not in df.index:
        df.loc[orf] = [None, None, None, None, None]

df

cl_description_proc

cl_description_proc = dict.fromkeys(df.index)
for key in cl_description_proc:
    cl_description_proc[key] = []
fh = open(proc_anno, 'r')
fh.readline()
for line in fh.readlines():
    ll = line.split('\t')
    name = f"{ll[1]} ({ll[2]})"
    li = ll[-1].strip().split(', ')
    for _ in li:
        if _=="SNR30": _ = "snR30"
        if _=="TP(UGG)O3": _ = "tP(UGG)O3"
        cl_description_proc[_].append(name)
for _ in cl_description_proc.keys():
    cl_description_proc[_] = " ".join(cl_description_proc[_])


cl_description_func = dict.fromkeys(df.index)
for key in cl_description_func:
    cl_description_func[key] = []
fh = open(fun_anno, 'r')
fh.readline()
for line in fh.readlines():
    ll = line.split('\t')
    name = f"{ll[1]} ({ll[2]})"
    li = ll[-1].strip().split(', ')
    for _ in li:
        if _=="SNR30": _ = "snR30"
        if _=="TP(UGG)O3": _ = "tP(UGG)O3"
        cl_description_func[_].append(name)
for _ in cl_description_func.keys():
    cl_description_func[_] = " ".join(cl_description_func[_])


df["GO_Function"] = [cl_description_func[_] for _ in df.index]
df["GO_Process"] = [cl_description_proc[_] for _ in df.index]

df

df.to_csv(f'{new_dir}/orf_info.csv')